# 基于 Qwen1.5-0.5b 的Question Answering系统
## 项目介绍
本项目旨在利用Qwen1.5-0.5b大语言模型，构建一个智能问答系统。​该系统不仅展示了大语言模型在专业领域文档理解和信息提取方面的能力，同时也为开发者提供了一个实践范例，说明如何结合预训练模型与特定领域知识，打造高效的问答解决方案。

## 环境准备
开发者拿到香橙派开发板后，首先需要进行硬件资源确认，镜像烧录及CANN和MindSpore版本的升级，才可运行该案例，具体如下：

开发板：香橙派AIpro 24G 20T开发板  
开发板镜像: Ubuntu镜像  
CANN Toolkit/Kernels：8.0.0.beta1  
MindSpore: 2.6.0  
MindNLP: 0.4.1  
Python: 3.9

### 镜像烧录
运行该案例需要烧录香橙派官网ubuntu镜像，烧录流程参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--镜像烧录](https://www.mindspore.cn/tutorials/zh-CN/r2.7.0rc1/orange_pi/environment_setup.html) 章节。

### CANN升级
CANN升级参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--CANN升级](https://www.mindspore.cn/tutorials/zh-CN/r2.7.0rc1/orange_pi/environment_setup.html)章节。

### MindSpore升级
MindSpore升级参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--MindSpore升级](https://www.mindspore.cn/tutorials/zh-CN/r2.7.0rc1/orange_pi/environment_setup.html)章节。

## 安装依赖包
本项目主要基于 Mindnlp 和 Mindspore 进行开发，除上述两个库外，需先安装相关依赖库


In [ ]:
!pip install pymupdf sentence-transformers faiss-cpu newspaper3k lxml[html_clean] jieba==0.42.1

## 加载Qwen1.5-0.5b模型
本项目使用小规模蒸馏模型

基于Mindspore框架和Mindnlp库开发安装还是很方便的

In [1]:
import mindspore
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer
from mindnlp.transformers import TextIteratorStreamer
from threading import Thread

# 加载tokenizer和模型
print("正在加载模型...")
tokenizer = AutoTokenizer.from_pretrained("/home/HwHiAiUser/Qwen1.5-0.5B-Chat", mirror="modelers", ms_dtype=mindspore.float16)
model = AutoModelForCausalLM.from_pretrained("/home/HwHiAiUser/Qwen1.5-0.5B-Chat", mirror="modelers", ms_dtype=mindspore.float16)
print("模型加载完成！")


/home/HwHiAiUser/.local/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/HwHiAiUser/.local/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/HwHiAiUser/.local/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/HwHiAiUser/.local/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
[WARNING] ME(7230:255085986443296,MainProcess):2025-09-11-15:30:28.629.326 [mindspore/context.py:1402]

正在加载模型...


Qwen2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


模型加载完成！


##  构建回答的流式输出函数

In [7]:
system_prompt = "你是个能回答问题的专家，需要根据用户的需求来具体解答"
def chat_with_model(user_input):
    """与模型进行单轮对话，支持流式输出"""
    # 构建消息格式
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_input}
    ]
    
    # 应用聊天模板
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="ms",
        tokenize=True
    )
    
    # 创建流式输出器
    streamer = TextIteratorStreamer(tokenizer, timeout=300, skip_prompt=True, skip_special_tokens=True)
    
    # 生成参数
    generate_kwargs = dict(
        input_ids=input_ids,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.9,
        temperature=0.1,
        num_beams=1,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # 在单独线程中启动生成
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    
    # 流式输出tokens
    full_response = ""
    for new_token in streamer:
        if '</s>' in new_token:  # 检查停止token
            break
        print(new_token, end="", flush=True)
        full_response += new_token
    
    return "回答完毕"

##  单轮LLM问答

In [8]:
query="你是谁"
chat_with_model(f"请回答：{query}")

我是来自阿里云的大规模语言模型，我叫通义千问。

'回答完毕'

##  连续多轮LLM问答

In [9]:
# 直接运行本单元格，然后在提示符连续输入问题；输入 q/quit/exit 结束。

from mindnlp.transformers import TextIteratorStreamer
from threading import Thread

# 基础生成参数（可保留默认值）
GEN_MAX_NEW_TOKENS = 512
GEN_TEMPERATURE = 0.7
GEN_TOP_P = 0.9

# 可选的系统提示词（如果不需要，置为空字符串）
SYSTEM_PROMPT = "你是一个专业的中文问答助手，请结合上下文，给出准确、简洁、有条理的回答。"

# 校验必要对象
if "tokenizer" not in globals() or "model" not in globals():
    raise RuntimeError("未检测到 tokenizer 或 model，请先运行上文加载模型的单元格。")

def generate_once(messages):
    """
    基于 MindNLP 的聊天模板与流式生成，输入完整 messages（包含历史），返回本轮回复字符串。
    messages: List[{'role': 'system'|'user'|'assistant', 'content': str}]
    """
    # 1) 构造输入（MindNLP: return_tensors='ms'）
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="ms",
        tokenize=True
    )

    # 2) 创建流式输出器（逐 token 返回字符串）
    streamer = TextIteratorStreamer(
        tokenizer=tokenizer,
        skip_prompt=True,
        skip_special_tokens=True
    )

    # 3) 组装生成参数
    generate_kwargs = dict(
        input_ids=input_ids,
        max_new_tokens=GEN_MAX_NEW_TOKENS,
        do_sample=True,
        temperature=GEN_TEMPERATURE,
        top_p=GEN_TOP_P,
        streamer=streamer,
        eos_token_id=getattr(tokenizer, "eos_token_id", None),
        pad_token_id=getattr(tokenizer, "pad_token_id", None),
    )

    # 4) 在线程中启动生成（主线程消费 streamer）
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    # 5) 流式打印并累计输出
    full_text = ""
    for piece in streamer:
        print(piece, end="", flush=True)
        full_text += piece
    print()  # 换行

    return full_text.strip()

# ========== 多轮对话主循环 ==========
history = []
if SYSTEM_PROMPT:
    history.append({"role": "system", "content": SYSTEM_PROMPT})

print("=== 连续问答已启动（输入 q/quit/exit 退出）===\n")
while True:
    try:
        user_text = input("你：").strip()
    except EOFError:
        print("\n[结束对话]")
        break

    if not user_text:
        continue
    if user_text.lower() in {"q", "quit", "exit"}:
        print("[结束对话]")
        break

    # 追加用户消息并生成
    history.append({"role": "user", "content": user_text})
    print("助手：", end="", flush=True)
    try:
        answer = generate_once(history)
    except Exception as e:
        print(f"[生成出错] {e}")
        # 出错时回滚上一轮的 user 消息，继续循环
        history.pop()
        continue

    # 记录助手消息，维护上下文
    history.append({"role": "assistant", "content": answer})


=== 连续问答已启动（输入 q/quit/exit 退出）===



你： 你是谁


助手：我是来自阿里云的大规模语言模型，我叫通义千问。


你： 介绍一下华为的主要业务


助手：华为是一家中国领先的通信设备和解决方案提供商。其主要业务包括智能手机、互联网设备（如路由器、交换机）、云计算服务以及大数据分析等。同时，华为还提供数据存储、网络安全、人工智能等领域的技术解决方案。


你： q


[结束对话]
